In [24]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [25]:
# Read the data from the spreadsheet
data = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Assignment 2 Files\price_response.csv")

In [26]:
# Extract relevant columns
product_lines = data["Product"].str.split(" ", n=1, expand=True)[0].unique()
versions = data["Product"].str.split(" ", n=2, expand=True)[2].unique()
coefficients = {(line, version): (intercept, sensitivity) for line, version, intercept, sensitivity in zip(data["Product"].str.split(" ", n=2, expand=True)[0], data["Product"].str.split(" ", n=2, expand=True)[2], data["Intercept"], data["Sensitivity"])}

### (a)

When TechEssentials initially launched 15 years ago, it had just one product line (the EvoTech
Pro Series computers) with two versions: Basic and Advanced. Apply the KKT conditions to
find the optimal prices of each of the two computers within this series, taking into account that
the Advanced version should have a higher price as compared to the Basic one. That is,

Maximize p1 (a1 − b1p1) + p2 (a1 − b1p2)

for some prices pi ≥ 0 and linear price response function coefficients ai ≥ 0 and bi ≥ 0 where
i = {1, 2} subject to price ordering and demand non-negativity constraints.

In [32]:
# (a) Optimal prices for EvoTech Pro Series Basic and Advanced versions
def optimal_prices_evotech(coefficients):
    # Extract coefficients for EvoTech Pro Series Basic and Advanced versions
    a1_basic = coefficients[('Line', 'Product 1')][0]
    b1_basic = coefficients[('Line', 'Product 1')][1]
    a1_advanced = coefficients[('Line', 'Product 2')][0]
    b1_advanced = coefficients[('Line', 'Product 2')][1]

    # Apply KKT conditions
    optimal_price_basic = a1_basic / (2 * b1_basic)
    optimal_price_advanced = a1_advanced / (2 * b1_advanced)

    print("Optimal prices for EvoTech Pro Series Basic and Advanced versions:")
    print("Basic:", optimal_price_basic)
    print("Advanced:", optimal_price_advanced)

### (b) 

Contrast the solution obtained from the previous question with the solution obtained from the
application of a projected gradient descent algorithm. Note that the constraint set is linear, so
the projection step can be solved as a quadratic program. Start with all prices initialized to zero,
a step size of 0.001, and a stopping criterion of 10−6

, what do you get as the optimal prices?

In [33]:
# (b) Projected Gradient Descent
def projected_gradient_descent(coefficients):
    # Initialization
    prices = {('Line', 'Product 1'): 0, ('Line', 'Product 2'): 0, ('Line', 'Product 3'): 0}
    step_size = 0.001
    stopping_criterion = 1e-6

    while True:
        # Compute gradients
        gradients = compute_gradients(prices, coefficients)

        # Update prices with gradient descent
        for key in prices:
            prices[key] += step_size * gradients[key]

        # Projection step to ensure non-negativity of prices
        for key in prices:
            prices[key] = max(prices[key], 0)

        # Check stopping criterion
        if max(abs(gradient) for gradient in gradients.values()) < stopping_criterion:
            break

    # Print optimal prices
    print("Optimal prices (Projected Gradient Descent):")
    for key, value in prices.items():
        print(f"{key}: {value}")

# Helper function to compute gradients
def compute_gradients(prices, coefficients):
    gradients = {}
    for key in prices:
        p = prices[key]
        gradients[key] = -2 * coefficients[key][1] * p + coefficients[key][0]
    return gradients

### (c)

Now consider all three product lines and the three versions within each product line. Assuming
that the Basic, Advanced, and Premium versions of the product should be increasing in price
within each product line, formulate and solve a quadratic optimization problem in Gurobi. What
is the optimal revenue suggested by the model?

### (d) 

Considering that prices should increase (i) within each product line (Basic, Advanced, Pre-
mium), and (ii) within the same version (e.g., a FusionBook Elite should be more expensive

than an InfiniteEdge Notebook which should be more expensive than an EvoTech Pro), modify
the previous quadratic optimization problem by adding new constraints to ensure this logic is
captured in the model. What is the optimal revenue TechEssentials can anticipate now?

In [34]:
# (c) and (d) Optimal prices for all product lines and versions
def optimal_prices_all(coefficients):
    model = gp.Model("Optimal_Prices_All")

    # Variables
    prices = {(p, v): model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"price_{p}_{v}") for p in product_lines for v in versions}

    # Objective function
    model.setObjective(gp.quicksum(prices[p, v] * (coefficients[p, v][0] - coefficients[p, v][1] * prices[p, v]) for p in product_lines for v in versions), sense=GRB.MAXIMIZE)

    # Constraints
    for p in product_lines:
        for i in range(len(versions) - 1):
            model.addConstr(prices[p, versions[i]] <= prices[p, versions[i + 1]])

    for i in range(len(product_lines) - 1):
        for v in versions:
            model.addConstr(prices[product_lines[i], v] <= prices[product_lines[i + 1], v])

    # Solve
    model.optimize()

    # Print results
    print("Optimal prices for all product lines and versions:")
    for p in product_lines:
        for v in versions:
            print(f"{p} - {v}: {prices[p, v].X}")

In [31]:
print("Keys in the coefficients dictionary:", coefficients.keys())


Keys in the coefficients dictionary: dict_keys([('Line', 'Product 1'), ('Line', 'Product 2'), ('Line', 'Product 3')])


In [35]:
# Example usage
optimal_prices_evotech(coefficients)
projected_gradient_descent(coefficients)
optimal_prices_all(coefficients)

Optimal prices for EvoTech Pro Series Basic and Advanced versions:
Basic: -8138.453042277662
Advanced: -4498.417684502101


### (e) 

What set of prices, from (c) or (d), do you think makes the most sense to implement in practice?

### (f) 

Do you believe something is missing such that the model does not faithfully represent reality.